In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary
from sklearn.model_selection import train_test_split

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections

In [2]:
# GPU Device
gpu_id = 3
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 3: True


# Arguments

In [3]:
source_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN_256/'
target_dir = '/media/data2/dataset/GAN_ImageData/StarGAN_128/'

In [4]:
pretrained = './log/style1/128/b1/checkpoint.pth.tar'
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b1' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 5000
start_epoch = 0
train_batch = 32
test_batch = 200
lr = 0.04
schedule = [200, 1500, 3000, 4500, 6000]
momentum = 0.9
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/style1/128/b1/to_star/cutmix2' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 4

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# sp
sp_alpha = 0.1
sp_beta = 0.1
fc_name = 'fc.'

# iterative training
feedback = 0
iter_time = [500, 1000, 1500, 2000, 2500, 3000, 3500, 4000]

# cutmix
cm_prob = 1
cm_prob_init = 0.99
cm_prob_low = 0.01

best_acc = 0

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

# Dataset

In [7]:
train_dir = os.path.join(target_dir, '100_shot')
source_train_dir = os.path.join(target_dir, '100_shot_style1_only')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
    transforms.RandomAffine(degrees=2, translate=(0.02, 0.02), scale=(0.98, 1.02), shear=2, fillcolor=(124,117,104)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
#     transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
source_train_loader = DataLoader(datasets.ImageFolder(source_train_dir, transform=val_aug),
                                batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_loader = DataLoader(datasets.ImageFolder(val_target_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_source_loader = DataLoader(datasets.ImageFolder(val_source_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [8]:
teacher_model = EfficientNet.from_name(model_name, num_classes=num_classes)
student_model = EfficientNet.from_name(model_name, num_classes=num_classes)

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    teacher_model.load_state_dict(torch.load(pretrained)['state_dict'])
    student_model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/style1/128/b1/checkpoint.pth.tar'


In [9]:
teacher_model.to('cuda')
student_model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in student_model.parameters())/1000000.0))

    Total params: 6.52M


In [10]:
for param in teacher_model.parameters():
    param.requires_grad = False
teacher_model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): GroupNorm(8, 32, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): GroupNorm(8, 32, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_bn2): GroupNorm(4, 16, ep

In [11]:
teacher_model_weights = {}
for name, param in teacher_model.named_parameters():
    teacher_model_weights[name] = param.detach()

In [12]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [13]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += 0.5 * torch.norm(param) ** 2
    return l2_cls

def reg_l2sp(model):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if not name.startswith(fc_name):
            sp_loss += 0.5 * torch.norm(param - teacher_model_weights[name]) ** 2
    return sp_loss

# Loss

In [14]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(student_model.parameters(), lr=lr, momentum=momentum)
# optimizer = optim.Adam(model.parameters(), weight_decay=0)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=4, total_epoch=50, after_scheduler=scheduler_cosine)

In [15]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    student_model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.', 'Source Loss', 'Source ACC.'])

# Train

In [16]:
def train(train_loader, source_train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda):
    student_model.train()
    torch.set_grad_enabled(True)
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()
    
    source_set = []
    for inputs, targets in source_train_loader:
        source_set.append((inputs, targets))
    
    bar = Bar('Processing', max=len(train_loader))
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)
        
        # cutmix
        source_inputs, source_targets = source_set[batch_idx]
        source_inputs, source_targets = source_inputs.cuda(), source_targets.cuda()

        prob_delta = cm_prob_init - cm_prob_low
        prob_step = epoch / (epochs+1) * prob_delta
        lam = cm_prob_init - prob_step

        rand_index = torch.randperm(inputs.size()[0]).cuda()
        st = source_targets[rand_index]
        tt = targets[rand_index]
        rand_index = rand_index[st == tt]
#         print(rand_index)

        bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), 1-lam)
        inputs[rand_index, :, bbx1:bbx2, bby1:bby2] = source_inputs[rand_index, :, bbx1:bbx2, bby1:bby2]
        lam = 1 - ((bbx2 - bbx1)*(bby2 - bby1) / (inputs.size()[-1] * inputs.size()[-2]))
        
        outputs = student_model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(student_model)
        loss_sp = reg_l2sp(student_model)
            
            
        # compute output
        loss = loss_main + sp_alpha*loss_sp + sp_beta*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx + 1,
                    size=len(train_loader),
                    data=data_time.val,
                    bt=batch_time.val,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,
                    )
        bar.next()
#         if batch_idx % 10 == 0:
        print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [17]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    torch.set_grad_enabled(False)

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    for batch_idx, (inputs, targets) in enumerate(val_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(model)
        loss_sp = reg_l2sp(model)
        loss = loss_main + sp_alpha*loss_sp + sp_beta*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                    batch=batch_idx + 1,
                    size=len(val_loader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,)
        bar.next()
    print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
         batch=batch_idx+1, size=len(val_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [18]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [ ]:
for epoch in range(start_epoch, epochs):
    # teacher feedback
    if epoch in iter_time:
        print("iterative training: feedback {}".format(epoch))
        teacher_model.load_state_dict(student_model.state_dict())
        teacher_model_weights = {}
        for name, param in teacher_model.named_parameters():
            teacher_model_weights[name] = param.detach()
    
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc = train(train_loader, source_train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda)
    
    if epoch % 20 == 0:
        test_loss, test_acc = test(val_target_loader, student_model, criterion, epoch, use_cuda)
        source_loss, source_acc = test(val_source_loader, student_model, criterion, epoch, use_cuda)

    
        logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc, source_loss, source_acc])
        is_best = test_acc > best_acc
        best_acc = max(test_acc, best_acc)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict' : student_model.state_dict(),
            'acc': test_acc,
            'best_acc': best_acc,
            'optimizer': optimizer.state_dict(),
        }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()


Epoch: [1 | 5000] LR: 0.040000
1/7 Data:5.283 | Batch:10.150 | Total:0:00:05 | ETA:0:00:33 | Loss:2.9266462326049805 | top1:62.5
2/7 Data:0.002 | Batch:3.462 | Total:0:00:08 | ETA:0:00:23 | Loss:2.8132256269454956 | top1:62.5
3/7 Data:0.011 | Batch:4.152 | Total:0:00:12 | ETA:0:00:18 | Loss:2.4492541551589966 | top1:53.125
4/7 Data:0.001 | Batch:3.505 | Total:0:00:16 | ETA:0:00:13 | Loss:2.0925260484218597 | top1:50.78125
5/7 Data:0.000 | Batch:3.772 | Total:0:00:20 | ETA:0:00:09 | Loss:1.9152160167694092 | top1:50.625
6/7 Data:0.016 | Batch:4.700 | Total:0:00:24 | ETA:0:00:05 | Loss:1.816526194413503 | top1:51.04166793823242


/home/cutz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


153/153 Data:0.003 | Batch:3.175 | Total:0:02:56 | ETA:0:00:00 | Loss:1.7017172264084122 | top1:50.000003814697266
39/39 Data:8.929 | Batch:9.836 | Total:0:01:20 | ETA:0:00:00 | Loss:1.676358672288748 | top1:50.0

Epoch: [2 | 5000] LR: 0.042400
1/7 Data:15.804 | Batch:19.018 | Total:0:00:08 | ETA:0:00:54 | Loss:1.6546599864959717 | top1:53.125
2/7 Data:0.001 | Batch:4.169 | Total:0:00:13 | ETA:0:00:33 | Loss:1.9660874605178833 | top1:54.6875
3/7 Data:0.006 | Batch:1.945 | Total:0:00:14 | ETA:0:00:20 | Loss:2.3439619541168213 | top1:52.083335876464844
4/7 Data:0.001 | Batch:4.885 | Total:0:00:19 | ETA:0:00:15 | Loss:2.7453699111938477 | top1:51.5625
5/7 Data:0.009 | Batch:1.763 | Total:0:00:21 | ETA:0:00:09 | Loss:3.1556586265563964 | top1:53.125
6/7 Data:0.117 | Batch:7.474 | Total:0:00:29 | ETA:0:00:05 | Loss:3.5786825021107993 | top1:52.083335876464844

Epoch: [3 | 5000] LR: 0.044800
1/7 Data:13.675 | Batch:17.317 | Total:0:00:08 | ETA:0:00:51 | Loss:6.4222941398620605 | top1:56.25
2

4/7 Data:0.001 | Batch:0.549 | Total:0:00:02 | ETA:0:00:02 | Loss:1233.3548583984375 | top1:66.40625
5/7 Data:0.021 | Batch:0.669 | Total:0:00:03 | ETA:0:00:02 | Loss:1189.7552612304687 | top1:68.125
6/7 Data:0.001 | Batch:4.882 | Total:0:00:08 | ETA:0:00:02 | Loss:1145.3432515462239 | top1:67.70833587646484

Epoch: [15 | 5000] LR: 0.073600
1/7 Data:17.076 | Batch:21.247 | Total:0:00:09 | ETA:0:00:59 | Loss:831.8116455078125 | top1:75.0
2/7 Data:0.005 | Batch:3.418 | Total:0:00:13 | ETA:0:00:34 | Loss:785.8314208984375 | top1:71.875
3/7 Data:0.003 | Batch:0.516 | Total:0:00:13 | ETA:0:00:19 | Loss:741.0885213216146 | top1:72.91667175292969
4/7 Data:0.013 | Batch:1.860 | Total:0:00:15 | ETA:0:00:12 | Loss:697.9679412841797 | top1:66.40625
5/7 Data:0.001 | Batch:4.369 | Total:0:00:19 | ETA:0:00:08 | Loss:656.5795288085938 | top1:63.125
6/7 Data:0.002 | Batch:3.988 | Total:0:00:23 | ETA:0:00:04 | Loss:617.0545705159506 | top1:63.54166793823242

Epoch: [16 | 5000] LR: 0.076000
1/7 Data:13.

1/7 Data:10.799 | Batch:15.681 | Total:0:00:07 | ETA:0:00:43 | Loss:261.0564270019531 | top1:50.0
2/7 Data:0.003 | Batch:5.079 | Total:0:00:12 | ETA:0:00:31 | Loss:276.6143798828125 | top1:54.6875
3/7 Data:0.001 | Batch:3.128 | Total:0:00:15 | ETA:0:00:21 | Loss:289.7340901692708 | top1:53.125
4/7 Data:0.005 | Batch:3.844 | Total:0:00:19 | ETA:0:00:15 | Loss:300.1317138671875 | top1:61.71875
5/7 Data:0.001 | Batch:3.257 | Total:0:00:22 | ETA:0:00:09 | Loss:307.9407653808594 | top1:65.625
6/7 Data:0.043 | Batch:5.113 | Total:0:00:27 | ETA:0:00:05 | Loss:313.2123107910156 | top1:65.625

Epoch: [28 | 5000] LR: 0.104800
1/7 Data:18.334 | Batch:22.408 | Total:0:00:09 | ETA:0:00:58 | Loss:349.5278015136719 | top1:56.25
2/7 Data:0.002 | Batch:3.632 | Total:0:00:13 | ETA:0:00:33 | Loss:364.5274963378906 | top1:46.875
3/7 Data:0.040 | Batch:3.826 | Total:0:00:17 | ETA:0:00:23 | Loss:383.34595743815106 | top1:57.29166793823242
4/7 Data:0.001 | Batch:3.973 | Total:0:00:20 | ETA:0:00:16 | Loss:404

...

4/7 Data:0.010 | Batch:0.363 | Total:0:00:01 | ETA:0:00:02 | Loss:0.03644256154075265 | top1:99.21875
5/7 Data:0.002 | Batch:0.372 | Total:0:00:02 | ETA:0:00:01 | Loss:0.043045106902718545 | top1:98.75
6/7 Data:0.009 | Batch:0.576 | Total:0:00:02 | ETA:0:00:01 | Loss:0.03980833354095618 | top1:98.95833587646484

Epoch: [4228 | 5000] LR: 0.000010
1/7 Data:1.859 | Batch:2.440 | Total:0:00:01 | ETA:0:00:07 | Loss:0.03577772527933121 | top1:100.0
2/7 Data:0.001 | Batch:0.441 | Total:0:00:01 | ETA:0:00:04 | Loss:0.036908261477947235 | top1:100.0
3/7 Data:0.003 | Batch:0.451 | Total:0:00:01 | ETA:0:00:03 | Loss:0.026068858957539003 | top1:100.0
4/7 Data:0.001 | Batch:0.404 | Total:0:00:02 | ETA:0:00:02 | Loss:0.025181640055961907 | top1:100.0
5/7 Data:0.009 | Batch:0.370 | Total:0:00:02 | ETA:0:00:02 | Loss:0.026567954663187267 | top1:99.375
6/7 Data:0.000 | Batch:0.301 | Total:0:00:03 | ETA:0:00:01 | Loss:0.024376457169031102 | top1:99.47917175292969

Epoch: [4229 | 5000] LR: 0.000010
1/7 D

2/7 Data:0.001 | Batch:0.378 | Total:0:00:01 | ETA:0:00:04 | Loss:0.028906184248626232 | top1:98.4375
3/7 Data:0.001 | Batch:0.511 | Total:0:00:01 | ETA:0:00:03 | Loss:0.05711015003422896 | top1:97.91667175292969
4/7 Data:0.001 | Batch:0.504 | Total:0:00:02 | ETA:0:00:02 | Loss:0.057384076062589884 | top1:97.65625
5/7 Data:0.001 | Batch:0.333 | Total:0:00:02 | ETA:0:00:02 | Loss:0.06340177692472934 | top1:97.5
6/7 Data:0.000 | Batch:0.435 | Total:0:00:03 | ETA:0:00:01 | Loss:0.06050902822365364 | top1:97.39583587646484

Epoch: [4241 | 5000] LR: 0.000010
1/7 Data:1.766 | Batch:2.403 | Total:0:00:01 | ETA:0:00:09 | Loss:0.06942103058099747 | top1:96.875
2/7 Data:0.002 | Batch:0.469 | Total:0:00:01 | ETA:0:00:05 | Loss:0.05733729340136051 | top1:98.4375
3/7 Data:0.001 | Batch:0.661 | Total:0:00:02 | ETA:0:00:04 | Loss:0.040199896010259785 | top1:98.95833587646484
4/7 Data:0.016 | Batch:0.559 | Total:0:00:03 | ETA:0:00:03 | Loss:0.037309932289645076 | top1:98.4375
5/7 Data:0.000 | Batch:0.

4/7 Data:0.016 | Batch:0.348 | Total:0:00:02 | ETA:0:00:02 | Loss:0.04294392955489457 | top1:98.4375
5/7 Data:0.004 | Batch:0.476 | Total:0:00:02 | ETA:0:00:02 | Loss:0.035932089202106 | top1:98.75
6/7 Data:0.003 | Batch:0.468 | Total:0:00:03 | ETA:0:00:01 | Loss:0.036080554593354464 | top1:98.95833587646484

Epoch: [4253 | 5000] LR: 0.000010
1/7 Data:1.943 | Batch:2.375 | Total:0:00:01 | ETA:0:00:07 | Loss:0.014127700589597225 | top1:100.0
2/7 Data:0.002 | Batch:0.360 | Total:0:00:01 | ETA:0:00:04 | Loss:0.030315767508000135 | top1:98.4375
3/7 Data:0.002 | Batch:0.500 | Total:0:00:02 | ETA:0:00:03 | Loss:0.07146038332333167 | top1:97.91667175292969
4/7 Data:0.001 | Batch:0.509 | Total:0:00:02 | ETA:0:00:02 | Loss:0.059324492467567325 | top1:98.4375
5/7 Data:0.000 | Batch:0.355 | Total:0:00:02 | ETA:0:00:02 | Loss:0.04985218774527311 | top1:98.75
6/7 Data:0.001 | Batch:0.422 | Total:0:00:03 | ETA:0:00:01 | Loss:0.053879938864459596 | top1:98.4375

Epoch: [4254 | 5000] LR: 0.000010
1/7 

6/7 Data:0.003 | Batch:0.343 | Total:0:00:02 | ETA:0:00:01 | Loss:0.060622127105792366 | top1:98.4375

Epoch: [4265 | 5000] LR: 0.000010
1/7 Data:1.537 | Batch:1.903 | Total:0:00:00 | ETA:0:00:06 | Loss:0.06570388376712799 | top1:96.875
2/7 Data:0.001 | Batch:0.513 | Total:0:00:01 | ETA:0:00:04 | Loss:0.0664195753633976 | top1:96.875
3/7 Data:0.001 | Batch:0.400 | Total:0:00:01 | ETA:0:00:03 | Loss:0.053450691824158035 | top1:97.91667175292969
4/7 Data:0.002 | Batch:0.452 | Total:0:00:02 | ETA:0:00:02 | Loss:0.05478607164695859 | top1:97.65625
5/7 Data:0.002 | Batch:0.358 | Total:0:00:02 | ETA:0:00:02 | Loss:0.07194557227194309 | top1:96.875
6/7 Data:0.001 | Batch:0.395 | Total:0:00:03 | ETA:0:00:01 | Loss:0.0607990485150367 | top1:97.39583587646484

Epoch: [4266 | 5000] LR: 0.000010
1/7 Data:1.764 | Batch:2.098 | Total:0:00:00 | ETA:0:00:06 | Loss:0.018049506470561028 | top1:100.0
2/7 Data:0.001 | Batch:0.395 | Total:0:00:01 | ETA:0:00:04 | Loss:0.04048645403236151 | top1:98.4375
3/7 

4/7 Data:0.015 | Batch:0.590 | Total:0:00:03 | ETA:0:00:03 | Loss:0.020550247514620423 | top1:100.0
5/7 Data:0.001 | Batch:0.512 | Total:0:00:03 | ETA:0:00:02 | Loss:0.02957197744399309 | top1:98.75
6/7 Data:0.001 | Batch:0.403 | Total:0:00:03 | ETA:0:00:01 | Loss:0.02574146236293018 | top1:98.95833587646484

Epoch: [4278 | 5000] LR: 0.000009
1/7 Data:2.851 | Batch:3.179 | Total:0:00:01 | ETA:0:00:08 | Loss:0.029190802946686745 | top1:100.0
2/7 Data:0.001 | Batch:0.523 | Total:0:00:01 | ETA:0:00:05 | Loss:0.022664147429168224 | top1:100.0
3/7 Data:0.001 | Batch:0.581 | Total:0:00:02 | ETA:0:00:04 | Loss:0.04576699870328108 | top1:97.91667175292969
4/7 Data:0.000 | Batch:0.615 | Total:0:00:02 | ETA:0:00:03 | Loss:0.05711893690750003 | top1:96.875
5/7 Data:0.011 | Batch:0.521 | Total:0:00:03 | ETA:0:00:02 | Loss:0.06599139384925365 | top1:96.875
6/7 Data:0.001 | Batch:0.673 | Total:0:00:04 | ETA:0:00:01 | Loss:0.0613112011924386 | top1:96.875

Epoch: [4279 | 5000] LR: 0.000009
1/7 Data:2